# Ariel Data Challenge 2025 - Advanced Pipeline

In [ ]:
!pip install --no-index --find-links=/kaggle/input/ariel-2024-pqdm pqdm > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import multiprocessing as mp
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import itertools
import glob
from dataclasses import dataclass
from typing import Tuple, Dict, Any, Optional, List

from tqdm import tqdm
from pqdm.threads import pqdm
from astropy.stats import sigma_clip
from scipy.optimize import minimize, curve_fit
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from scipy.signal import savgol_filter
from sklearn.metrics import mean_squared_error
from scipy import interpolate
from scipy.ndimage import median_filter

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Master toggle - set False to use original pipeline
USE_ADVANCED_PIPELINE = True

ROOT_PATH = "/kaggle/input/ariel-data-challenge-2025"
MODE = "test"
EPS = 1e-12

In [ ]:
# Original Config (preserved for compatibility)
class Config:
    DATA_PATH = '/kaggle/input/ariel-data-challenge-2025'
    DATASET = "test"

    SCALE = 0.946
    SIGMA = 0.00056
    
    CUT_INF = 39
    CUT_SUP = 321
    
    SENSOR_CONFIG = {
        "AIRS-CH0": {
            "raw_shape": [11250, 32, 356],
            "calibrated_shape": [1, 32, CUT_SUP - CUT_INF],
            "linear_corr_shape": (6, 32, 356),
            "dt_pattern": (0.1, 4.5), 
            "binning": 30
        },
        "FGS1": {
            "raw_shape": [135000, 32, 32],
            "calibrated_shape": [1, 32, 32],
            "linear_corr_shape": (6, 32, 32),
            "dt_pattern": (0.1, 0.1),
            "binning": 30 * 12
        }
    }
    
    MODEL_PHASE_DETECTION_SLICE = slice(30, 140)
    MODEL_OPTIMIZATION_DELTA = 11
    MODEL_POLYNOMIAL_DEGREE = 3
    
    N_JOBS = 3

# Advanced Config
@dataclass
class AdvancedConfig:
    # Calibration / binning
    time_bin_factor_fgs: int = 360
    time_bin_factor_airs: int = 30
    
    # Extraction
    fgs_aperture_radius_px: int = 5
    bg_annulus: Tuple[int, int] = (7, 10)
    
    # GP/systematics
    gp_kernel: str = "matern32"
    gp_use_shared_hyperparams: bool = True
    gp_white_jitter: float = 1e-6
    
    # Sigma calibration
    beta_bin_sizes: Tuple[int, int] = (1, 8)
    sigma_floor_fgs: float = 2e-4
    sigma_floor_airs: float = 2e-4
    
    # Wavelength smoothing
    wl_smooth_strength: float = 0.05
    
    # Multi-visit fusion
    fuse_visits: bool = True
    
    # Data paths
    data_path: str = '/kaggle/input/ariel-data-challenge-2025'
    dataset: str = "test"
    n_jobs: int = 3

config = Config()
adv_config = AdvancedConfig()

## Original Pipeline Components (Preserved)

In [ ]:
def _phase_detector_signal(signal, cfg):
    sl = cfg.MODEL_PHASE_DETECTION_SLICE
    min_idx = int(np.argmin(signal[sl])) + sl.start
    s1 = signal[:min_idx]; s2 = signal[min_idx:]
    
    if s1.size < 3 or s2.size < 3:
        return 0, len(signal) - 1
    
    g1 = np.gradient(s1); g1_max = np.max(g1) if np.size(g1) else 0.0
    g2 = np.gradient(s2); g2_max = np.max(g2) if np.size(g2) else 0.0
    
    if g1_max != 0:
        g1 /= g1_max
    if g2_max != 0:
        g2 /= g2_max
    
    phase1 = int(np.argmin(g1))
    phase2 = int(np.argmax(g2)) + min_idx
    
    return phase1, phase2

def estimate_sigma_fgs(preprocessed_data, cfg):
    sig_rel = []
    delta = cfg.MODEL_OPTIMIZATION_DELTA
    eps = 1e-12
    
    for single in preprocessed_data:
        air_white = savgol_filter(single[:, 1:].mean(axis=1), 20, 2, mode='nearest')
        p1, p2 = _phase_detector_signal(air_white, cfg)
        p1 = max(delta, p1)
        p2 = min(len(air_white) - delta - 1, p2)

        fgs = single[:, 0]
        oot = (fgs[: p1 - delta] if p1 - delta > 0 else np.empty(0, fgs.dtype))
        if p2 + delta < fgs.size:
            oot = np.concatenate([oot, fgs[p2 + delta :]])
        inn = fgs[p1 + delta : max(p1 + delta, p2 - delta)]

        if oot.size == 0 or inn.size == 0:
            sig_rel.append(np.nan); continue

        n_oot, n_in = len(oot), len(inn)
        var_oot = np.nanvar(oot, ddof=1)
        var_in  = np.nanvar(inn, ddof=1)
        oot_mean = float(np.nanmean(oot)) if np.isfinite(np.nanmean(oot)) else float(np.nanmean(fgs))
        sigma_rel = np.sqrt(var_oot / max(n_oot,1) + var_in / max(n_in,1)) / max(oot_mean, eps)
        sig_rel.append(sigma_rel)

    s = np.asarray(sig_rel, dtype=float)
    mask = np.isfinite(s) & (s > 0)
    med = float(np.nanmedian(s[mask])) if mask.any() else 1.0

    k = np.ones_like(s)
    if med > 0 and np.isfinite(med):
        k[mask] = np.sqrt(s[mask] / med)
    
    k = np.clip(k, 0.85, 1.30) 
    
    return k * cfg.SIGMA * 1.04

def estimate_sigma_air(preprocessed_data, cfg):
    sig_rel = []
    delta = cfg.MODEL_OPTIMIZATION_DELTA
    eps = 1e-12

    for single in preprocessed_data:
        white = np.nanmean(single[:, 1:], axis=1)
        white_s = savgol_filter(white, 20, 2, mode='nearest')

        p1, p2 = _phase_detector_signal(white_s, cfg)
        p1 = max(delta, p1)
        p2 = min(len(white) - delta - 1, p2)

        oot_left = white[: p1 - delta] if p1 - delta > 0 else np.empty(0, white.dtype)
        oot_right = white[p2 + delta :] if (p2 + delta) < white.size else np.empty(0, white.dtype)
        oot = np.concatenate([oot_left, oot_right]) if (oot_left.size + oot_right.size) else oot_left
        inn = white[p1 + delta : max(p1 + delta, p2 - delta)]

        if oot.size == 0 or inn.size == 0:
            sig_rel.append(np.nan); continue

        n_oot, n_in = len(oot), len(inn)
        var_oot = np.nanvar(oot, ddof=1)
        var_in  = np.nanvar(inn, ddof=1)
        oot_mean = float(np.nanmean(oot)) if np.isfinite(np.nanmean(oot)) else float(np.nanmean(white))

        sigma_rel = np.sqrt(var_oot / max(n_oot,1) + var_in / max(n_in,1)) / max(oot_mean, eps)
        sig_rel.append(sigma_rel)

    s = np.asarray(sig_rel, dtype=float)
    mask = np.isfinite(s) & (s > 0)
    med = float(np.nanmedian(s[mask])) if mask.any() else 1.0

    k = np.ones_like(s)
    if med > 0 and np.isfinite(med):
        k[mask] = np.sqrt(s[mask] / med)
    
    k = np.clip(k, 0.92, 1.22)

    return k * cfg.SIGMA * 1.04

In [ ]:
class SignalProcessor:
    def __init__(self, config):
        self.cfg = config
        self.adc_info = pd.read_csv(f"{self.cfg.DATA_PATH}/adc_info.csv")
        self.planet_ids = pd.read_csv(f'{self.cfg.DATA_PATH}/{self.cfg.DATASET}_star_info.csv', index_col='planet_id').index.astype(int)

    def _apply_linear_corr(self, linear_corr, signal):
        coeffs = np.flip(linear_corr, axis=0)
        x = signal.astype(np.float64, copy=False)
        out = np.empty_like(x, dtype=np.float64)
        out[...] = coeffs[0]
        for k in range(1, coeffs.shape[0]):
            np.multiply(out, x, out=out)
            out += coeffs[k]

        return out.astype(signal.dtype, copy=False)

    def _calibrate_single_signal(self, planet_id, sensor):
        sensor_cfg = self.cfg.SENSOR_CONFIG[sensor]

        signal = pd.read_parquet(f"{self.cfg.DATA_PATH}/{self.cfg.DATASET}/{planet_id}/{sensor}_signal_0.parquet").to_numpy()
        dark = pd.read_parquet(f"{self.cfg.DATA_PATH}/{self.cfg.DATASET}/{planet_id}/{sensor}_calibration_0/dark.parquet").to_numpy()
        dead = pd.read_parquet(f"{self.cfg.DATA_PATH}/{self.cfg.DATASET}/{planet_id}/{sensor}_calibration_0/dead.parquet").to_numpy()
        flat = pd.read_parquet(f"{self.cfg.DATA_PATH}/{self.cfg.DATASET}/{planet_id}/{sensor}_calibration_0/flat.parquet").to_numpy()
        linear_corr = pd.read_parquet(f"{self.cfg.DATA_PATH}/{self.cfg.DATASET}/{planet_id}/{sensor}_calibration_0/linear_corr.parquet").values.astype(np.float64).reshape(sensor_cfg["linear_corr_shape"])

        signal = signal.reshape(sensor_cfg["raw_shape"])
        gain = self.adc_info[f"{sensor}_adc_gain"].iloc[0]
        offset = self.adc_info[f"{sensor}_adc_offset"].iloc[0]
        signal = signal / gain + offset

        hot = sigma_clip(dark, sigma=5, maxiters=5).mask

        if sensor == "AIRS-CH0":
            signal = signal[:, :, self.cfg.CUT_INF : self.cfg.CUT_SUP]
            linear_corr = linear_corr[:, :, self.cfg.CUT_INF : self.cfg.CUT_SUP]
            dark = dark[:, self.cfg.CUT_INF : self.cfg.CUT_SUP]
            dead = dead[:, self.cfg.CUT_INF : self.cfg.CUT_SUP]
            flat = flat[:, self.cfg.CUT_INF : self.cfg.CUT_SUP]
            hot = hot[:, self.cfg.CUT_INF : self.cfg.CUT_SUP]


        if sensor == "FGS1":
            y0, y1, x0, x1 = 10, 22, 10, 22
            signal = signal[:, y0:y1, x0:x1]
            dark   = dark[y0:y1, x0:x1]
            dead   = dead[y0:y1, x0:x1]
            flat   = flat[y0:y1, x0:x1]
            linear_corr = linear_corr[:, y0:y1, x0:x1]
            hot    = hot[y0:y1, x0:x1]

        np.maximum(signal, 0, out=signal)

        if sensor == "FGS1":
            signal = self._apply_linear_corr(linear_corr, signal)
        elif sensor == "AIRS-CH0":
            sl = (slice(None), slice(10, 22), slice(None))
            signal[sl] = self._apply_linear_corr(linear_corr[:, 10:22, :], signal[sl])
        else:
            signal = self._apply_linear_corr(linear_corr, signal)

        base_dt, increment = sensor_cfg["dt_pattern"]
        even_scale = base_dt
        odd_scale  = base_dt + increment

        signal[::2] -= dark * even_scale
        signal[1::2] -= dark * odd_scale

        
        return signal

    def _preprocess_calibrated_signal(self, calibrated_signal, sensor):
        sensor_cfg = self.cfg.SENSOR_CONFIG[sensor]
        binning = sensor_cfg["binning"]

        if sensor == "AIRS-CH0":
            signal_roi = calibrated_signal[:, 10:22, :]
        elif sensor == "FGS1":
            signal_roi = calibrated_signal[:, 10:22, 10:22]
            signal_roi = signal_roi.reshape(signal_roi.shape[0], -1)
        
        mean_signal = np.nanmean(signal_roi, axis=1)

        cds_signal = mean_signal[1::2] - mean_signal[0::2]

        n_bins = cds_signal.shape[0] // binning
        binned = np.array([
            cds_signal[j*binning : (j+1)*binning].mean(axis=0) 
            for j in range(n_bins)
        ])

        if sensor == "AIRS-CH0":
            q_lo = np.nanpercentile(binned, 5.0, axis=1, keepdims=True)
            q_hi = np.nanpercentile(binned, 95.0, axis=1, keepdims=True)
            np.clip(binned, q_lo, q_hi, out=binned)

        if sensor == "FGS1":
            binned = binned.reshape((binned.shape[0], 1))

        if sensor == "AIRS-CH0":
            var = np.nanvar(binned, axis=0, ddof=1)
            med = np.nanmedian(var)

            safe_var = np.where(~np.isfinite(var) | (var <= 0), med if (np.isfinite(med) and med > 0) else 1.0, var)
            w = 1.0 / safe_var

            lo, hi = np.nanpercentile(w, 5.0), np.nanpercentile(w, 95.0)
            if np.isfinite(lo) and np.isfinite(hi) and lo < hi:
                w = np.clip(w, lo, hi)

            M = binned.shape[1]
            s = np.nansum(w)
            if np.isfinite(s) and s > 0:
                w = w * (M / s)
            else:
                w = np.ones_like(w)

            binned *= w[None, :]


        return binned

    def _process_planet_sensor(self, args):
        planet_id, sensor = args['planet_id'], args['sensor']
        calibrated = self._calibrate_single_signal(planet_id, sensor)
        preprocessed = self._preprocess_calibrated_signal(calibrated, sensor)
        return preprocessed

    def process_all_data(self):
        args_fgs1 = [dict(planet_id=planet_id, sensor="FGS1") for planet_id in self.planet_ids]
        preprocessed_fgs1 = pqdm(args_fgs1, self._process_planet_sensor, n_jobs=self.cfg.N_JOBS)

        args_airs_ch0 = [dict(planet_id=planet_id, sensor="AIRS-CH0") for planet_id in self.planet_ids]
        preprocessed_airs_ch0 = pqdm(args_airs_ch0, self._process_planet_sensor, n_jobs=self.cfg.N_JOBS)

        preprocessed_signal = np.concatenate(
            [np.stack(preprocessed_fgs1), np.stack(preprocessed_airs_ch0)], axis=2
        )
        
        return preprocessed_signal

In [ ]:
class TransitModel:
    def __init__(self, config):
        self.cfg = config

    def _phase_detector(self, signal):
        search_slice = self.cfg.MODEL_PHASE_DETECTION_SLICE
        min_index = np.argmin(signal[search_slice]) + search_slice.start
        
        signal1 = signal[:min_index]
        signal2 = signal[min_index:]

        grad1 = np.gradient(signal1)
        grad1 /= grad1.max()
        
        grad2 = np.gradient(signal2)
        grad2 /= grad2.max()

        phase1 = np.argmin(grad1)
        phase2 = np.argmax(grad2) + min_index

        return phase1, phase2
    
    def _objective_function(self, s, signal, phase1, phase2):
        delta = self.cfg.MODEL_OPTIMIZATION_DELTA
        power = self.cfg.MODEL_POLYNOMIAL_DEGREE

        if phase1 - delta <= 0 or phase2 + delta >= len(signal) or phase2 - delta - (phase1 + delta) < 5:
            delta = 2

        y = np.concatenate([
            signal[: phase1 - delta],
            signal[phase1 + delta : phase2 - delta] * (1 + s),
            signal[phase2 + delta :]
        ])
        x = np.arange(len(y))

        coeffs = np.polyfit(x, y, deg=power)
        poly = np.poly1d(coeffs)
        error = np.abs(poly(x) - y).mean()
        
        return error

    def predict(self, single_preprocessed_signal):
        signal_1d = single_preprocessed_signal[:, 1:].mean(axis=1)
        signal_1d = savgol_filter(signal_1d, 20, 2, mode='nearest')
        
        phase1, phase2 = self._phase_detector(signal_1d)

        phase1 = max(self.cfg.MODEL_OPTIMIZATION_DELTA, phase1)
        phase2 = min(len(signal_1d) - self.cfg.MODEL_OPTIMIZATION_DELTA - 1, phase2)    

        result = minimize(
            fun=self._objective_function,
            x0=[0.0001],
            args=(signal_1d, phase1, phase2),
            method="Nelder-Mead"
        )
        
        return result.x[0]

    def predict_all(self, preprocessed_signals):
        predictions = [
            self.predict(preprocessed_signal)
            for preprocessed_signal in tqdm(preprocessed_signals)
        ]
        return np.array(predictions) * self.cfg.SCALE

## Advanced Pipeline Components

In [ ]:
# Utility functions
def robust_mean(x, axis=None):
    return np.nanmean(x, axis=axis)

def robust_std(x, axis=None):
    return np.nanstd(x, axis=axis, ddof=1) + EPS

def clip_sigma(s, s_min, s_max=None):
    s = np.maximum(s, s_min)
    if s_max is not None:
        s = np.minimum(s, s_max)
    return s

In [ ]:
class AdvancedCalibrator:
    """Advanced calibration with full pipeline"""
    
    def __init__(self, config: AdvancedConfig):
        self.cfg = config
        self.adc_info = pd.read_csv(f"{self.cfg.data_path}/adc_info.csv")
        
    def load_calibration_frames(self, planet_id, sensor, visit=0):
        base_path = f"{self.cfg.data_path}/{self.cfg.dataset}/{planet_id}/{sensor}_calibration_{visit}"
        
        dark = pd.read_parquet(f"{base_path}/dark.parquet").to_numpy()
        dead = pd.read_parquet(f"{base_path}/dead.parquet").to_numpy()
        flat = pd.read_parquet(f"{base_path}/flat.parquet").to_numpy()
        linear_corr = pd.read_parquet(f"{base_path}/linear_corr.parquet").to_numpy()
        read_noise = pd.read_parquet(f"{base_path}/read.parquet").to_numpy()
        
        return {
            'dark': dark,
            'dead': dead,
            'flat': flat,
            'linear_corr': linear_corr,
            'read_noise': read_noise
        }
    
    def apply_adc_conversion(self, signal, sensor):
        """Convert from uint16 to float64 with ADC parameters"""
        gain = self.adc_info[f"{sensor}_adc_gain"].iloc[0]
        offset = self.adc_info[f"{sensor}_adc_offset"].iloc[0]
        return signal.astype(np.float64) / gain + offset
    
    def apply_nonlinearity_correction(self, signal, linear_corr):
        """Apply polynomial linearity correction"""
        coeffs = np.flip(linear_corr, axis=0)
        x = signal.astype(np.float64)
        result = np.zeros_like(x)
        result[...] = coeffs[0]
        
        for k in range(1, coeffs.shape[0]):
            result *= x
            result += coeffs[k]
            
        return result
    
    def mask_bad_pixels(self, signal, dead, hot_mask):
        """Mask and interpolate dead/hot pixels"""
        bad_mask = dead | hot_mask
        
        if bad_mask.any():
            # For 2D images, use median of neighbors
            if signal.ndim == 3:
                for t in range(signal.shape[0]):
                    frame = signal[t]
                    if bad_mask.any():
                        frame[bad_mask] = median_filter(frame, size=3)[bad_mask]
            
        return signal
    
    def apply_flat_fielding(self, signal, flat):
        """Apply flat field correction"""
        flat_safe = np.where(flat > 0.5, flat, 1.0)
        return signal / flat_safe
    
    def calibrate_signal(self, planet_id, sensor, visit=0):
        """Full calibration pipeline"""

        # Load raw signal
        signal_path = f"{self.cfg.data_path}/{self.cfg.dataset}/{planet_id}/{sensor}_signal_{visit}.parquet"
        signal = pd.read_parquet(signal_path).to_numpy()

        # Reshape based on sensor
        if sensor == "FGS1":
            signal = signal.reshape(135000, 32, 32)
        else:  # AIRS-CH0
            signal = signal.reshape(11250, 32, 356)

        # Load calibration frames
        cal_frames = self.load_calibration_frames(planet_id, sensor, visit)

        # ADC conversion
        signal = self.apply_adc_conversion(signal, sensor)

        # CROP EVERYTHING FIRST (matching original pipeline)
        if sensor == "FGS1":
            # Crop signal and all calibration frames to 12x12 region
            signal = signal[:, 10:22, 10:22]
            cal_frames['dark'] = cal_frames['dark'][10:22, 10:22]
            cal_frames['dead'] = cal_frames['dead'][10:22, 10:22]
            cal_frames['flat'] = cal_frames['flat'][10:22, 10:22]
            # Reshape and crop linear correction
            linear_corr = cal_frames['linear_corr'].reshape(6, 32, 32)[:, 10:22, 10:22]
        elif sensor == "AIRS-CH0":
            # Crop to valid spectral range
            signal = signal[:, :, 39:321]
            cal_frames['dark'] = cal_frames['dark'][:, 39:321]
            cal_frames['dead'] = cal_frames['dead'][:, 39:321]
            cal_frames['flat'] = cal_frames['flat'][:, 39:321]
            # Reshape and crop linear correction
            linear_corr = cal_frames['linear_corr'].reshape(6, 32, 356)[:, :, 39:321]

        # Ensure signal is non-negative after ADC
        np.maximum(signal, 0, out=signal)

        # Apply non-linearity correction
        if sensor == "FGS1":
            signal = self.apply_nonlinearity_correction(signal, linear_corr)
        elif sensor == "AIRS-CH0":
            # Only apply to rows 10:22 as in original
            sl = (slice(None), slice(10, 22), slice(None))
            signal[sl] = self.apply_nonlinearity_correction(signal[sl], linear_corr[:, 10:22, :])

        # Dark subtraction with proper scaling
        if sensor == "FGS1":
            dt = 0.1
            signal -= cal_frames['dark'] * dt
        else:
            dt_even = 0.1
            dt_odd = 4.6
            signal[::2] -= cal_frames['dark'] * dt_even
            signal[1::2] -= cal_frames['dark'] * dt_odd

        # Hot pixel detection (on cropped dark)
        hot_mask = sigma_clip(cal_frames['dark'], sigma=5, maxiters=5).mask

        # Mask bad pixels
        signal = self.mask_bad_pixels(signal, cal_frames['dead'], hot_mask)

        # Flat fielding
        signal = self.apply_flat_fielding(signal, cal_frames['flat'])

        return signal

In [ ]:
class AdvancedExtractor:
    """Optimal extraction for both instruments"""
    
    def __init__(self, config: AdvancedConfig):
        self.cfg = config
    
    def cds_and_bin(self, frames, bin_factor):
        """Correlated Double Sampling and temporal binning"""
        # CDS: difference consecutive pairs
        cds = frames[1::2] - frames[0::2]
        
        # Temporal binning
        n_bins = len(cds) // bin_factor
        binned = np.zeros((n_bins,) + cds.shape[1:])
        
        for i in range(n_bins):
            binned[i] = np.mean(cds[i*bin_factor:(i+1)*bin_factor], axis=0)
            
        return binned
    
    def centroid(self, image):
        """Calculate flux-weighted centroid"""
        y, x = np.mgrid[:image.shape[0], :image.shape[1]]
        
        total = np.sum(image)
        if total == 0:
            return image.shape[0] // 2, image.shape[1] // 2
        
        cx = np.sum(x * image) / total
        cy = np.sum(y * image) / total
        
        return cy, cx
    
    def fgs_aperture_photometry(self, frames, radius=5, bg_annulus=(7, 10)):
        """Optimal aperture photometry for FGS1"""
        n_frames = frames.shape[0]
        photometry = np.zeros(n_frames)
        
        for i in range(n_frames):
            frame = frames[i]
            
            # Find centroid
            cy, cx = self.centroid(frame)
            
            # Create aperture mask
            y, x = np.mgrid[:frame.shape[0], :frame.shape[1]]
            dist = np.sqrt((x - cx)**2 + (y - cy)**2)
            
            # Aperture mask
            aperture = dist <= radius
            
            # Background annulus mask
            bg_mask = (dist >= bg_annulus[0]) & (dist <= bg_annulus[1])
            
            # Calculate background
            if bg_mask.any():
                bg_median = np.median(frame[bg_mask])
            else:
                bg_median = 0
            
            # Aperture photometry
            photometry[i] = np.sum(frame[aperture] - bg_median)
        
        return photometry
    
    def track_airs_trace(self, frames):
        """Track spectral trace for AIRS"""
        n_frames, n_rows, n_cols = frames.shape
        trace_centers = np.zeros((n_frames, n_cols))
        
        # Use median frame as reference
        ref_frame = np.median(frames, axis=0)
        
        for col in range(n_cols):
            col_profile = ref_frame[:, col]
            
            # Find peak row (trace center)
            if np.sum(col_profile) > 0:
                center = np.sum(np.arange(n_rows) * col_profile) / np.sum(col_profile)
            else:
                center = n_rows // 2
            
            trace_centers[:, col] = center
        
        return trace_centers
    
    def horne_optimal_extraction(self, frames, trace_centers=None, width=5):
        """Horne optimal extraction for AIRS"""
        n_frames, n_rows, n_cols = frames.shape
        
        if trace_centers is None:
            trace_centers = self.track_airs_trace(frames)
        
        extracted = np.zeros((n_frames, n_cols))
        
        for t in range(n_frames):
            for col in range(n_cols):
                # Extract around trace center
                center = int(trace_centers[t, col])
                
                # Define extraction window
                row_min = max(0, center - width)
                row_max = min(n_rows, center + width + 1)
                
                # Simple sum for now (can be improved with proper weights)
                col_data = frames[t, row_min:row_max, col]
                
                # Background subtraction
                bg_rows = np.concatenate([
                    np.arange(max(0, row_min-3), row_min),
                    np.arange(row_max, min(n_rows, row_max+3))
                ])
                
                if len(bg_rows) > 0:
                    bg = np.median(frames[t, bg_rows, col])
                else:
                    bg = 0
                    
                extracted[t, col] = np.sum(col_data - bg)
        
        return extracted

In [ ]:
class CommonModeRemoval:
    """Common mode systematics removal"""
    
    @staticmethod
    def compute_common_mode(white_light, window=51, poly=3):
        """Compute common mode from white light curve"""
        # Smooth the white light curve
        if len(white_light) > window:
            smooth = savgol_filter(white_light, window, poly, mode='nearest')
        else:
            smooth = white_light
        
        # Common mode is the residual pattern
        common_mode = white_light / (smooth + EPS)
        
        return common_mode
    
    @staticmethod
    def apply_common_mode(data, common_mode):
        """Apply common mode correction to data"""
        if data.ndim == 1:
            return data / (common_mode + EPS)
        else:
            # For 2D data (time, wavelength)
            return data / (common_mode[:, None] + EPS)

In [ ]:
class TransitFitter:
    """Physics-based transit fitting with GP systematics"""
    
    def __init__(self, config: AdvancedConfig):
        self.cfg = config
    
    def simple_transit_model(self, time, depth, t0, duration):
        """Simple box-car transit model"""
        transit = np.ones_like(time)
        in_transit = np.abs(time - t0) < duration / 2
        transit[in_transit] = 1.0 - depth
        return transit
    
    def detect_transit_phase(self, light_curve):
        """Detect transit ingress and egress"""
        light_curve = np.nan_to_num(light_curve, nan=np.nanmedian(light_curve), posinf=np.nanmedian(light_curve), neginf=np.nanmedian(light_curve))

        # Smooth the light curve
        if len(light_curve) > 51:
            smooth = savgol_filter(light_curve, 51, 3, mode='nearest')
        else:
            smooth = light_curve
        
        # Find minimum (mid-transit)
        mid_transit = np.argmin(smooth)
        
        # Find ingress and egress by gradient
        grad = np.gradient(smooth)
        
        # Ingress: steepest negative gradient before mid-transit
        if mid_transit > 10:
            ingress = np.argmin(grad[:mid_transit])
        else:
            ingress = 0
        
        # Egress: steepest positive gradient after mid-transit  
        if mid_transit < len(grad) - 10:
            egress = mid_transit + np.argmax(grad[mid_transit:])
        else:
            egress = len(light_curve) - 1
        
        return ingress, mid_transit, egress
    
    def fit_transit_depth(self, time, flux, star_params=None):
        """Fit transit depth with simple detrending"""
        
        # Ensure flux is finite
        flux_clean = np.nan_to_num(flux, nan=np.nanmedian(flux), posinf=np.nanmedian(flux), neginf=np.nanmedian(flux))
        
        # If flux is all NaN or zero, return default values
        if np.all(np.isnan(flux)) or np.all(flux_clean == 0):
            return 0.001, 0.0001  # Default depth and sigma
        
        # Detect transit timing
        ingress, mid_transit, egress = self.detect_transit_phase(flux_clean)
        
        # Define in and out of transit
        oot_mask = (time < ingress - 5) | (time > egress + 5)
        in_transit_mask = (time >= ingress + 5) & (time <= egress - 5)
        
        if not oot_mask.any() or not in_transit_mask.any():
            # Fallback if phase detection fails
            n = len(flux_clean)
            oot_mask = np.zeros(n, dtype=bool)
            oot_mask[:n//4] = True
            oot_mask[3*n//4:] = True
            in_transit_mask = ~oot_mask
        
        # Estimate depth from ratio
        oot_level = np.median(flux_clean[oot_mask]) if oot_mask.any() else np.median(flux_clean)
        in_transit_level = np.median(flux_clean[in_transit_mask]) if in_transit_mask.any() else np.min(flux_clean)
        
        # Ensure levels are finite and positive
        if not np.isfinite(oot_level) or oot_level <= 0:
            oot_level = np.nanmedian(flux_clean[flux_clean > 0]) if np.any(flux_clean > 0) else 1.0
        if not np.isfinite(in_transit_level):
            in_transit_level = oot_level * 0.99  # Default 1% depth
        
        depth = 1.0 - (in_transit_level / oot_level)
        depth = np.clip(depth, 0, 0.1)  # Reasonable bounds
        
        # Ensure depth is finite
        if not np.isfinite(depth):
            depth = 0.001  # Default depth
        
        # Estimate uncertainty from out-of-transit scatter
        if oot_mask.any():
            oot_std = robust_std(flux_clean[oot_mask])
        else:
            oot_std = robust_std(flux_clean)
            
        # Ensure std is finite and positive
        if not np.isfinite(oot_std) or oot_std <= 0:
            oot_std = 0.001 * oot_level  # Default to 0.1% of level
            
        n_in = np.sum(in_transit_mask)
        n_out = np.sum(oot_mask)
        
        # Propagate uncertainty
        if n_in > 0 and n_out > 0:
            sigma_depth = oot_std * np.sqrt(1.0/n_in + 1.0/n_out) / oot_level
        else:
            sigma_depth = oot_std / oot_level
        
        # Ensure sigma is finite and positive
        if not np.isfinite(sigma_depth) or sigma_depth <= 0:
            sigma_depth = 0.0001  # Default uncertainty
        
        return depth, sigma_depth
    
    def fit_with_gp(self, time, flux, star_params=None):
        """Fit transit with simple GP-like detrending"""
        
        # Initial depth estimate
        depth, sigma = self.fit_transit_depth(time, flux, star_params)
        
        # Simple iterative detrending
        for _ in range(3):
            # Build transit model
            ingress, mid_transit, egress = self.detect_transit_phase(flux)
            transit = self.simple_transit_model(time, depth, mid_transit, egress - ingress)
            
            # Detrend
            residuals = flux / (transit + EPS)
            if len(residuals) > 101:
                trend = savgol_filter(residuals, 101, 3, mode='nearest')
            else:
                trend = np.ones_like(residuals)
            
            # Correct flux
            flux_detrended = flux / (trend + EPS)
            
            # Re-fit depth
            depth, sigma = self.fit_transit_depth(time, flux_detrended, star_params)
        
        return depth, sigma

In [ ]:
class SigmaCalibrator:
    """Advanced uncertainty estimation"""
    
    def __init__(self, config: AdvancedConfig):
        self.cfg = config
    
    def beta_inflation(self, residuals, bin_sizes=(1, 8)):
        """Calculate beta factor for red noise"""
        rms_unbinned = robust_std(residuals)
        
        if len(residuals) < bin_sizes[1] * 10:
            return 1.0
        
        # Bin the residuals
        n_bins = len(residuals) // bin_sizes[1]
        binned = np.array([np.mean(residuals[i*bin_sizes[1]:(i+1)*bin_sizes[1]]) 
                          for i in range(n_bins)])
        
        rms_binned = robust_std(binned)
        
        # Beta factor
        expected_reduction = 1.0 / np.sqrt(bin_sizes[1])
        actual_reduction = rms_binned / rms_unbinned
        
        beta = actual_reduction / expected_reduction
        beta = np.clip(beta, 1.0, 10.0)
        
        return beta
    
    def calibrate_sigma(self, sigma_raw, flux, residuals=None):
        """Calibrate uncertainty estimate"""
        
        # Apply floor
        sigma = clip_sigma(sigma_raw, self.cfg.sigma_floor_airs)
        
        # Beta inflation if residuals available
        if residuals is not None:
            beta = self.beta_inflation(residuals)
            sigma *= beta
        
        # SNR-based scaling
        snr = np.median(flux) / robust_std(flux)
        if snr < 10:
            sigma *= 1.5
        elif snr > 100:
            sigma *= 0.9
        
        return sigma

In [ ]:
class VisitFusion:
    """Multi-visit fusion utilities"""
    
    @staticmethod
    def get_visits(planet_id, data_path, dataset):
        """Check for multiple visits"""
        planet_path = f"{data_path}/{dataset}/{planet_id}"
        
        # Look for multiple signal files
        fgs_files = glob.glob(f"{planet_path}/FGS1_signal_*.parquet")
        n_visits = len(fgs_files)
        
        if n_visits > 1:
            return list(range(n_visits))
        return [0]
    
    @staticmethod
    def fuse_depths(depths, sigmas):
        """Inverse variance weighted fusion"""
        depths = np.array(depths)
        sigmas = np.array(sigmas)
        
        # Inverse variance weights
        weights = 1.0 / (sigmas**2 + EPS)
        
        # Weighted mean
        fused_depth = np.sum(weights * depths) / np.sum(weights)
        
        # Combined uncertainty
        fused_sigma = np.sqrt(1.0 / np.sum(weights))
        
        return fused_depth, fused_sigma

In [ ]:
class WavelengthSmoother:
    """Cross-wavelength smoothing"""
    
    @staticmethod
    def smooth_spectrum(spectrum, strength=0.1):
        """Gentle smoothing across wavelengths"""
        if strength == 0:
            return spectrum
        
        # Simple Gaussian smoothing
        window = int(5 * (1 + strength * 10))
        if window % 2 == 0:
            window += 1
        
        if len(spectrum) > window:
            smoothed = savgol_filter(spectrum, window, 3, mode='nearest')
            # Blend with original
            return (1 - strength) * spectrum + strength * smoothed
        
        return spectrum

In [ ]:
class AdvancedPipeline:
    """Main advanced processing pipeline"""
    
    def __init__(self, config: AdvancedConfig):
        self.cfg = config
        self.calibrator = AdvancedCalibrator(config)
        self.extractor = AdvancedExtractor(config)
        self.transit_fitter = TransitFitter(config)
        self.sigma_calibrator = SigmaCalibrator(config)
        self.star_info = pd.read_csv(f"{config.data_path}/{config.dataset}_star_info.csv", 
                                     index_col='planet_id')
    
    def process_visit(self, planet_id, visit=0):
        """Process single visit"""
        
        # Get star parameters
        star_params = self.star_info.loc[planet_id].to_dict()
        
        # Calibrate signals
        fgs_cal = self.calibrator.calibrate_signal(planet_id, "FGS1", visit)
        airs_cal = self.calibrator.calibrate_signal(planet_id, "AIRS-CH0", visit)
        
        # Apply CDS and binning
        fgs_cds = self.extractor.cds_and_bin(fgs_cal, self.cfg.time_bin_factor_fgs)
        airs_cds = self.extractor.cds_and_bin(airs_cal, self.cfg.time_bin_factor_airs)
        
        # Extract photometry
        fgs_flux = self.extractor.fgs_aperture_photometry(fgs_cds, 
                                                          self.cfg.fgs_aperture_radius_px,
                                                          self.cfg.bg_annulus)
        
        # Optimal extraction for AIRS
        airs_flux = self.extractor.horne_optimal_extraction(airs_cds)
        
        # Common mode removal
        fgs_white = fgs_flux  # FGS is already white light
        airs_white = np.mean(airs_flux, axis=1)
        
        # Use FGS as common mode
        common_mode = CommonModeRemoval.compute_common_mode(fgs_white)
        
        # Apply common mode to AIRS
        airs_corrected = CommonModeRemoval.apply_common_mode(airs_flux, common_mode)
        
        # Fit transits
        time = np.arange(len(fgs_white))
        
        # FGS1 depth
        depth_fgs, sigma_fgs = self.transit_fitter.fit_with_gp(time, fgs_white, star_params)
        
        # AIRS depths (per channel)
        n_channels = airs_corrected.shape[1]
        depths_airs = np.zeros(n_channels)
        sigmas_airs = np.zeros(n_channels)
        
        for i in range(n_channels):
            depths_airs[i], sigmas_airs[i] = self.transit_fitter.fit_with_gp(
                time, airs_corrected[:, i], star_params
            )
        
        # Calibrate uncertainties
        sigma_fgs = self.sigma_calibrator.calibrate_sigma(sigma_fgs, fgs_white)
        
        for i in range(n_channels):
            sigmas_airs[i] = self.sigma_calibrator.calibrate_sigma(
                sigmas_airs[i], airs_corrected[:, i]
            )
        
        # Apply wavelength smoothing to AIRS
        depths_airs = WavelengthSmoother.smooth_spectrum(depths_airs, 
                                                         self.cfg.wl_smooth_strength)
        
        # Validation: ensure all outputs are finite and positive
        if not np.isfinite(depth_fgs) or depth_fgs <= 0:
            depth_fgs = 0.001  # Default depth
        if not np.isfinite(sigma_fgs) or sigma_fgs <= 0:
            sigma_fgs = 0.0001  # Default sigma
            
        # Validate AIRS depths and sigmas
        for i in range(len(depths_airs)):
            if not np.isfinite(depths_airs[i]) or depths_airs[i] <= 0:
                depths_airs[i] = 0.001  # Default depth
            if not np.isfinite(sigmas_airs[i]) or sigmas_airs[i] <= 0:
                sigmas_airs[i] = 0.0001  # Default sigma
        
        # Final validation - replace any remaining NaN/Inf
        depth_fgs = np.nan_to_num(depth_fgs, nan=0.001, posinf=0.001, neginf=0.001)
        sigma_fgs = np.nan_to_num(sigma_fgs, nan=0.0001, posinf=0.0001, neginf=0.0001)
        depths_airs = np.nan_to_num(depths_airs, nan=0.001, posinf=0.001, neginf=0.001)
        sigmas_airs = np.nan_to_num(sigmas_airs, nan=0.0001, posinf=0.0001, neginf=0.0001)
        
        # Ensure positive values
        depth_fgs = np.abs(depth_fgs) if depth_fgs != 0 else 0.001
        sigma_fgs = np.abs(sigma_fgs) if sigma_fgs != 0 else 0.0001
        depths_airs = np.abs(depths_airs)
        depths_airs[depths_airs == 0] = 0.001
        sigmas_airs = np.abs(sigmas_airs)
        sigmas_airs[sigmas_airs == 0] = 0.0001
        
        return depth_fgs, sigma_fgs, depths_airs[:282], sigmas_airs[:282]
    
    def process_planet(self, planet_id):
        """Process planet with potential multi-visit fusion"""
        
        # Check for multiple visits
        visits = VisitFusion.get_visits(planet_id, self.cfg.data_path, self.cfg.dataset)
        
        if len(visits) == 1:
            # Single visit
            return self.process_visit(planet_id, visits[0])
        
        # Multi-visit fusion
        all_depths_fgs = []
        all_sigmas_fgs = []
        all_depths_airs = []
        all_sigmas_airs = []
        
        for visit in visits:
            d_fgs, s_fgs, d_airs, s_airs = self.process_visit(planet_id, visit)
            all_depths_fgs.append(d_fgs)
            all_sigmas_fgs.append(s_fgs)
            all_depths_airs.append(d_airs)
            all_sigmas_airs.append(s_airs)
        
        # Fuse FGS
        depth_fgs, sigma_fgs = VisitFusion.fuse_depths(all_depths_fgs, all_sigmas_fgs)
        
        # Fuse each AIRS channel
        n_channels = len(all_depths_airs[0])
        depths_airs = np.zeros(n_channels)
        sigmas_airs = np.zeros(n_channels)
        
        for i in range(n_channels):
            channel_depths = [d[i] for d in all_depths_airs]
            channel_sigmas = [s[i] for s in all_sigmas_airs]
            depths_airs[i], sigmas_airs[i] = VisitFusion.fuse_depths(channel_depths, 
                                                                     channel_sigmas)
        
        # Final validation for fused values
        depth_fgs = np.nan_to_num(depth_fgs, nan=0.001, posinf=0.001, neginf=0.001)
        sigma_fgs = np.nan_to_num(sigma_fgs, nan=0.0001, posinf=0.0001, neginf=0.0001)
        depths_airs = np.nan_to_num(depths_airs, nan=0.001, posinf=0.001, neginf=0.001)
        sigmas_airs = np.nan_to_num(sigmas_airs, nan=0.0001, posinf=0.0001, neginf=0.0001)
        
        # Ensure positive
        depth_fgs = np.abs(depth_fgs) if depth_fgs != 0 else 0.001
        sigma_fgs = np.abs(sigma_fgs) if sigma_fgs != 0 else 0.0001
        depths_airs = np.abs(depths_airs)
        depths_airs[depths_airs == 0] = 0.001
        sigmas_airs = np.abs(sigmas_airs)
        sigmas_airs[sigmas_airs == 0] = 0.0001
        
        return depth_fgs, sigma_fgs, depths_airs, sigmas_airs

## Neural Network Models

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, dim, p=0.2):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim)
        self.bn1 = nn.BatchNorm1d(dim)
        self.fc2 = nn.Linear(dim, dim)
        self.bn2 = nn.BatchNorm1d(dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.fc1(x)))
        out = self.dropout(out)
        out = self.bn2(self.fc2(out))
        return self.relu(out + identity)


class ResNetMLP(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=32, output_dim=1, num_blocks=3, dropout_rate=0.2):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.blocks = nn.Sequential(*[ResidualBlock(hidden_dim, p=dropout_rate) for _ in range(num_blocks)])
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.blocks(x)
        x = self.output_layer(x)
        return x


class ResNetMLP2(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=128, output_dim=282, num_blocks=3, dropout_rate=0.2):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.blocks = nn.Sequential(*[ResidualBlock(hidden_dim, p=dropout_rate) for _ in range(num_blocks)])
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.input_layer(x)
        x = self.blocks(x)
        x = self.output_layer(x)
        return x

In [ ]:
class SubmissionGenerator:
    def __init__(self, config):
        self.cfg = config
        self.sample_submission = pd.read_csv("/kaggle/input/ariel-data-challenge-2025/sample_submission.csv", index_col="planet_id")

    def create(self, predictions1, predictions2, predictions, sigma_fgs=None, sigma_air=None):
        planet_ids = self.sample_submission.index
        n_mu = self.sample_submission.shape[1] // 2

        preds = np.asarray(predictions, dtype=float).reshape(-1)
        mu = np.tile(preds.reshape(-1, 1), (1, n_mu))
        mu = np.clip(mu, 0, None)

        sigmas = np.full_like(mu, self.cfg.SIGMA, dtype=float)
        if sigma_fgs is not None:
            sigma_fgs = np.asarray(sigma_fgs, dtype=float).reshape(-1)
            sigmas[:, 0] = np.clip(sigma_fgs, 1e-6, 0.1)
        if sigma_air is not None:
            sigma_air = np.asarray(sigma_air, dtype=float)
            # Handle both 1D and 2D arrays
            if sigma_air.ndim == 1:
                sigma_air = sigma_air.reshape(-1, 1)
                sigmas[:, 1:] = np.clip(sigma_air, 1e-6, 0.1)
            else:
                # sigma_air is already (n_planets, n_channels)
                sigmas[:, 1:283] = np.clip(sigma_air, 1e-6, 0.1)

        submission_df = pd.DataFrame(
            np.concatenate([mu, sigmas], axis=1),
            columns=self.sample_submission.columns,
            index=planet_ids
        )
        submission_df.iloc[:, 1:283] = predictions2
        submission_df.iloc[:, 0] = predictions1

        submission_df.to_csv("submission.csv")
        return submission_df

## Main Execution

In [ ]:
# Load star information
StarInfo = pd.read_csv(ROOT_PATH + f"/{MODE}_star_info.csv")
StarInfo["planet_id"] = StarInfo["planet_id"].astype(int)
PlanetIds = StarInfo["planet_id"].tolist()
StarInfo = StarInfo.set_index("planet_id")

print(f"Processing {len(PlanetIds)} planets...")
print(f"Using {'ADVANCED' if USE_ADVANCED_PIPELINE else 'ORIGINAL'} pipeline")

In [ ]:
if USE_ADVANCED_PIPELINE:
    # Use advanced pipeline
    print("Running advanced pipeline...")
    
    # Initialize pipeline
    pipeline = AdvancedPipeline(adv_config)
    
    # Process all planets
    all_depths_fgs = []
    all_sigmas_fgs = []
    all_depths_airs = []
    all_sigmas_airs = []
    
    for planet_id in tqdm(PlanetIds, desc="Processing planets"):
        depth_fgs, sigma_fgs, depths_airs, sigmas_airs = pipeline.process_planet(planet_id)
        
        all_depths_fgs.append(depth_fgs)
        all_sigmas_fgs.append(sigma_fgs)
        all_depths_airs.append(depths_airs)
        all_sigmas_airs.append(sigmas_airs)
    
    # Convert to arrays
    predictions1 = np.array(all_depths_fgs).reshape(-1, 1)
    predictions2 = np.array(all_depths_airs)
    sigma_fgs_vec = np.array(all_sigmas_fgs)
    sigma_air_vec = np.array(all_sigmas_airs)
    
    # Overall predictions (for compatibility)
    predictions = np.mean(np.concatenate([predictions1, 
                                          np.mean(predictions2, axis=1, keepdims=True)], 
                                          axis=1), axis=1)
    
else:
    # Use original pipeline
    print("Running original pipeline...")
    
    # Process signals
    signal_processor = SignalProcessor(config)
    preprocessed_data = signal_processor.process_all_data()
    
    # Transit model predictions
    model = TransitModel(config)
    predictions = model.predict_all(preprocessed_data)
    
    # Neural network predictions
    predictions_df = pd.DataFrame(predictions, columns=["transit_depth"])
    
    # Prepare features for NN
    input_df = StarInfo.copy()
    pred_series = predictions_df["transit_depth"].set_axis(input_df.index, copy=False)
    input_df.insert(0, "transit_depth", (pred_series * 10000).to_numpy())
    
    features = ["transit_depth", "Rs", "i"]
    X = input_df[features].values.astype("float32")
    X_tensor = torch.tensor(X, dtype=torch.float32)
    
    # FGS1 predictions
    model_fgs = ResNetMLP(num_blocks=80, dropout_rate=0.2)
    model_fgs.load_state_dict(torch.load("/kaggle/input/fgs1/pytorch/default/1/best_model.pth"))
    model_fgs.eval()
    
    with torch.no_grad():
        predictions1 = model_fgs(X_tensor).numpy() / 10000
    
    # AIRS predictions
    model_airs = ResNetMLP2(num_blocks=80, dropout_rate=0.3)
    model_airs.load_state_dict(torch.load("/kaggle/input/airs/pytorch/default/1/best_model_airs.pth"))
    model_airs.eval()
    
    with torch.no_grad():
        predictions2 = model_airs(X_tensor).numpy() / 10000
    
    # Estimate uncertainties
    sigma_fgs_vec = estimate_sigma_fgs(preprocessed_data, config)
    sigma_air_vec = estimate_sigma_air(preprocessed_data, config)

In [ ]:
# Generate submission
submission_generator = SubmissionGenerator(config)
submission = submission_generator.create(predictions1, predictions2, predictions, 
                                         sigma_fgs=sigma_fgs_vec, sigma_air=sigma_air_vec)

print("\nSubmission shape:", submission.shape)
print("First row preview:")
print(submission.iloc[0, :10])  # Show first 10 columns
print("\nSubmission saved to submission.csv")